In [1]:
import netCDF4 as nc
from netCDF4 import Dataset
import xarray as xr
import time

In [2]:
mesh_mask = xr.open_dataset('/Volumes/Untitled/Cropped_DataTransfer/masks/meshMask.nc')

/opt/anaconda3/envs/geomar/lib/python3.7/site-packages/xarray/backends/plugins.py:68: RuntimeWarning: Engine 'rasterio' loading failed:
dlopen(/opt/anaconda3/envs/geomar/lib/python3.7/site-packages/rasterio/_base.cpython-37m-darwin.so, 0x0002): Symbol not found: _ZSTD_compressBound
  Referenced from: <BFC06822-0FF5-3311-8ECD-FB1AE1C722BC> /opt/anaconda3/envs/geomar/lib/libgdal.30.dylib
  Expected in:     <446AECE7-E150-30BD-8CBE-87E9B8DB9C5B> /opt/anaconda3/envs/geomar/lib/libblosc.1.21.5.dylib
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [3]:
mask = xr.open_dataset('/Volumes/Untitled/Cropped_DataTransfer/masks/tmask.nc')

In [4]:
# calculate volume for each grid cell

dx = mesh_mask.e1t 
dy = mesh_mask.e2t
grid_cell_area = dx * dy # m2

In [5]:
gridded_mesh = grid_cell_area.isel(t=0)

In [6]:
# apply mask to weighted grid cells

ocean_area = gridded_mesh * mask

In [7]:
# slice the european area

ocean_area_sliced = ocean_area.sel(x=slice(520,595),y=slice(330,435),z=0,time=0)

In [8]:
# slice location S

ocean_area_sliced_dp = ocean_area_sliced.isel(x=slice(58,60),y=slice(41,43))

In [9]:
mld_base = xr.open_dataset('/Volumes/Untitled/Cropped_DataTransfer/46dData/mldBase70.nc')
mld_oae = xr.open_dataset('/Volumes/Untitled/Cropped_DataTransfer/46dData/mldOAE70.nc')

In [10]:
alk_base = xr.open_dataset('/Volumes/Untitled/Cropped_DataTransfer/46dData/alkBase70.nc')
alk_oae = xr.open_dataset('/Volumes/Untitled/Cropped_DataTransfer/46dData/alkOAE70.nc')

In [11]:
dic_base = xr.open_dataset('/Volumes/Untitled/Cropped_DataTransfer/46dData/dicBase70.nc')
dic_oae = xr.open_dataset('/Volumes/Untitled/Cropped_DataTransfer/46dData/dicOAE70.nc')

In [12]:
datasets = [alk_base, mld_base, alk_oae, mld_oae, dic_base, dic_oae]

In [13]:
data_new = []

for x in datasets:
    x = x.sel(time_counter=slice('2090','2100'))
    data_new.append(x)

In [14]:
alk_base = data_new[0]
mld_base = data_new[1]
alk_oae = data_new[2]
mld_oae = data_new[3]
dic_base = data_new[4]
dic_oae = data_new[5]

In [15]:
def slicing_alk(variable, mld):
    layers_alk = []
    mld_bool = (variable.deptht < mld.somxl010).to_dataset(name='mld') 
    variable = variable * mld_bool.mld
    variable = variable.where(variable)
    variable = variable.fillna(0)
    variable = variable * 1.025   # potential seawater density
    for x in list((range(len(variable.deptht)))):
        if x == 0:
            n_variable = variable.ALK.isel(deptht=x) * variable.deptht[x] 
            layers_alk.append(n_variable)
        else:
            n_variable = variable.ALK.isel(deptht=x) * (variable.deptht[x] - variable.deptht[x-1])
            layers_alk.append(n_variable)
    weighted_var = sum(layers_alk) / mld.somxl010
    weighted_var = weighted_var.to_dataset(name='ALK')
    return(weighted_var)

In [16]:
def slicing_dic(variable, mld):
    layers_dic = []
    mld_bool = (variable.deptht < mld.somxl010).to_dataset(name='mld') 
    variable = variable * mld_bool.mld
    variable = variable.where(variable)
    variable = variable.fillna(0)
    variable = variable * 1.025   # potential seawater density
    for x in list((range(len(variable.deptht)))):
        if x == 0:
            n_variable = variable.DIC.isel(deptht=x) * variable.deptht[x] 
            layers_dic.append(n_variable)
        else:
            n_variable = variable.DIC.isel(deptht=x) * (variable.deptht[x] - variable.deptht[x-1])
            layers_dic.append(n_variable)
    weighted_var = sum(layers_dic) / mld.somxl010
    weighted_var = weighted_var.to_dataset(name='DIC')
    return(weighted_var)

In [17]:
%time

alk_base_weighted = slicing_alk(alk_base, mld_base)
alk_oae_weighted = slicing_alk(alk_oae, mld_oae)
dic_base_weighted = slicing_dic(dic_base, mld_base)
dic_oae_weighted = slicing_dic(dic_oae, mld_oae)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.11 µs


In [18]:
datasets = [alk_base_weighted.ALK, alk_oae_weighted.ALK, dic_base_weighted.DIC, dic_oae_weighted.DIC]
var_new = []

for x in datasets:
    sliced_x = x * ocean_area_sliced  # baseline
    regridded_x = sliced_x.sum(['x', 'y']) / ocean_area_sliced.sum(['x', 'y'])
    var_new.append(regridded_x)

In [19]:
alk_base_regridded = var_new[0].tmask.rename("ALK")
alk_oae_regridded = var_new[1].tmask.rename("ALK")
dic_base_regridded = var_new[2].tmask.rename("DIC")
dic_oae_regridded = var_new[3].tmask.rename("DIC")

In [20]:
location_datasets = [alk_base.ALK, alk_oae.ALK, dic_base.DIC, dic_oae.DIC, mld_base.somxl010, mld_oae.somxl010]

In [21]:
location_regrid = []

for x in location_datasets:
    x = x.isel(x=slice(58,60),y=slice(41,43))
    location_regrid.append(x)

In [22]:
alk_base_dp = location_regrid[0].to_dataset(name='ALK')
alk_oae_dp = location_regrid[1].to_dataset(name='ALK')
dic_base_dp = location_regrid[2].to_dataset(name='DIC')
dic_oae_dp = location_regrid[3].to_dataset(name='DIC')
mld_base = location_regrid[4].to_dataset(name='somxl010')
mld_oae = location_regrid[5].to_dataset(name='somxl010')

In [23]:
%time

alk_base_weighted_dp = slicing_alk(alk_base_dp, mld_base)
alk_oae_weighted_dp = slicing_alk(alk_oae_dp, mld_oae)
dic_base_weighted_dp = slicing_dic(dic_base_dp, mld_base)
dic_oae_weighted_dp = slicing_dic(dic_oae_dp, mld_oae)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 8.11 µs


In [24]:
datasets_location = [alk_base_weighted_dp.ALK, alk_oae_weighted_dp.ALK, dic_base_weighted_dp.DIC, dic_oae_weighted_dp.DIC]
var_new_location = []

for x in datasets_location:
    sliced_x = x * ocean_area_sliced_dp  # baseline
    regridded_x = sliced_x.sum(['x', 'y']) / ocean_area_sliced_dp.sum(['x', 'y'])
    var_new_location.append(regridded_x)

In [25]:
regridded_alk_base_dp = var_new_location[0].tmask.rename("ALK")
regridded_alk_oae_dp = var_new_location[1].tmask.rename("ALK")
dic_base_regridded_dp = var_new_location[2].tmask.rename("DIC")
dic_oae_regridded_dp = var_new_location[3].tmask.rename("DIC")

In [26]:
# alk_base_weighted.to_netcdf('/Users/chiaraciscato/Desktop/geomar/OAEseasonality/weighting/ssp3-7.0/alkBaseLineplot.nc')
# alk_oae_weighted.to_netcdf('/Users/chiaraciscato/Desktop/geomar/OAEseasonality/weighting/ssp3-7.0/alkOAELineplot.nc')
# dic_base_weighted.to_netcdf('/Users/chiaraciscato/Desktop/geomar/OAEseasonality/weighting/ssp3-7.0/dicBaseLineplot.nc')
# dic_oae_weighted.to_netcdf('/Users/chiaraciscato/Desktop/geomar/OAEseasonality/weighting/ssp3-7.0/dicOAELineplot.nc')

In [27]:
# alk_base_regridded.to_netcdf('/Users/chiaraciscato/Desktop/geomar/OAEseasonality/weighting/ssp3-7.0/alkBaseMap.nc')
# alk_oae_regridded.to_netcdf('/Users/chiaraciscato/Desktop/geomar/OAEseasonality/weighting/ssp3-7.0/alkOAEMap.nc')
# dic_base_regridded.to_netcdf('/Users/chiaraciscato/Desktop/geomar/OAEseasonality/weighting/ssp3-7.0/dicBaseMap.nc')
# dic_oae_regridded.to_netcdf('/Users/chiaraciscato/Desktop/geomar/OAEseasonality/weighting/ssp3-7.0/diOAEMap.nc')

In [28]:
# alk_base_regridded.to_netcdf('/Users/chiaraciscato/Desktop/geomar/OAEseasonality/weighting/ssp3-7.0/alkBaseMapS.nc')
# alk_oae_regridded.to_netcdf('/Users/chiaraciscato/Desktop/geomar/OAEseasonality/weighting/ssp3-7.0/alkOAEMapS.nc')
# dic_base_regridded.to_netcdf('/Users/chiaraciscato/Desktop/geomar/OAEseasonality/weighting/ssp3-7.0/dicBaseMapS.nc')
# dic_oae_regridded.to_netcdf('/Users/chiaraciscato/Desktop/geomar/OAEseasonality/weighting/ssp3-7.0/diOAEMapS.nc')